## TODO: PDF Page Classification Task

Objective:
Implement the `classify_page` function within the given code structure to classify each page of a PDF file into one of three categories based on its content and readability.

Code Structure:
The main function `classify_all_pages` is already implemented, but if necessary you are allowed to change its implementation. Your task is to complete the `classify_page` function.

Input:
- A PDF file path (the function should be able to handle various PDF files)

Output:
- A list of integers, where each integer represents the class of a page in the PDF

Classification Categories:

0: Machine-readable / searchable
   - Pages with text that can be directly extracted and searched within the PDF

1: Non-machine readable but OCR-able
   - Pages containing text that isn't directly extractable but can be recognized through OCR
   - Essentially, these are pages with visible text but stored as images

2: Non-machine readable and not OCR-able
   - Pages without any recognizable text
   - This may include pages with only images, complex graphics, or blank pages

Task:
1. Implement the `classify_page` function:
   - Input: A single page object (PdfReader.PageObject)
   - Output: An integer (0, 1, or 2) representing the page's class

2. The function should analyze the content of the page and determine its class based on the categories described above.

3. Ensure your implementation is robust and can handle various types of PDF content.

Requirements:
1. The function should work with different PDF files, not just a specific one.
2. Implement methods to distinguish between the three categories accurately.
3. Handle potential exceptions or edge cases (e.g., corrupted pages, mixed content types on a single page).
4. Optimize for both accuracy and processing speed, as the function will be called for each page in the PDF.
5. You are allowed to use up to 40GB of GPU VRAM if necessary for your implementation.

Additional Considerations:
- You may use additional libraries if needed, but ensure they are imported properly.
- Provide clear comments in your code to explain the classification logic.

Testing:
- Test your implementation with various types of PDFs to ensure its robustness and generalizability.
- The main script provides a way to test your implementation on a file named "grouped_documents.pdf".


In [2]:
from typing import List
from PyPDF2 import PdfReader, PdfWriter
import pytesseract
from PIL import Image
import fitz  # PyMuPDF
import io
pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

def classify_all_pages(input_pdf: str) -> List[int]:
    """
    Analyze all pages in the input PDF and determine the class of the pdf page.

    Args:
    input_pdf (str): The file path of the input PDF.

    Returns:
    List[int]: A list of classes for each page. 
            0: machine-readable
            1: non-machine readable but OCR-able
            2: non-machine readable and not OCR-able
    """
    reader = PdfReader(input_pdf)
    classes = []
    
    try:
        doc = fitz.open(input_pdf)
        for page_number in range(len(reader.pages)):
            current_page = doc.load_page(page_number)
            page_class = classify_page(current_page, page_number)
            classes.append(page_class)
    except Exception as e:
        print(f"An error occurred: {e}")
    
    return classes

def classify_page(page: 'fitz.Page', page_number: int) -> int:
    """
    Determine the class of the pdf page.

    Args:
    page (fitz.Page): A single page from a PDF.
    page_number (int): The page number in the document.

    Returns:
    int: The page is 
        0: machine-readable
        1: non-machine readable but OCR-able
        2: non-machine readable and not OCR-able
    """
    # Get page dimensions
    rect = page.rect
    height = rect.height
    margin = 0.10 * height  # 15% margin top and bottom

    # Define the area to extract text from (middle 70% of the page)
    middle_rect = fitz.Rect(rect.x0, rect.y0 + margin, rect.x1, rect.y1 - margin)

    # Extract text from the middle area of the page
    text = page.get_text("text", clip=middle_rect)
    if len(text) > 0:
        return 0

    # If no machine-readable text, convert the page to an image and apply OCR
    try:
        # Convert the specified region of the PDF page to an image
        pix = page.get_pixmap(clip=middle_rect)
        img = Image.open(io.BytesIO(pix.tobytes()))
        # Use pytesseract to perform OCR on the image
        ocr_text = pytesseract.image_to_string(img)
        if ocr_text and ocr_text.strip():
            return 1
    except Exception as e:
        print(f"OCR error on page {page_number}: {e}")

    # If OCR also fails to recognize any text, classify as not OCR-able
    return 2

# Usage
input_pdf: str = "grouped_documents.pdf"
page_classes: List[int] = classify_all_pages(input_pdf)
print(f"Classes for each page: {page_classes}")


Classes for each page: [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
